# Libraries

In [7]:
import os
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Constants

In [9]:
FOLDER_PATH = './gas_consumption/'
FILE_EXSTENSION = '.json'

## Showing Results

In [36]:
all_files = os.listdir(FOLDER_PATH)

# Filter files based on the pattern Name*_collaborator.json and Name*_manager.json
collaborator_files = [file for file in all_files if '_collaborator' in file]
manager_files = [file for file in all_files if '_manager' in file]

# Process files in pairs (collaborator and manager)
for collaborator_file, manager_file in zip(collaborator_files, manager_files):
    collaborator_path = os.path.join(FOLDER_PATH, collaborator_file)
    manager_path = os.path.join(FOLDER_PATH, manager_file)

    # Load JSON data from files
    with open(collaborator_path, 'r') as f_collaborator, open(manager_path, 'r') as f_manager:
        collaborator_data = json.load(f_collaborator)
        manager_data = json.load(f_manager)

    for device in collaborator_data.keys():
        collaborator_data[device]['retrieve_fee'] = collaborator_data[device]['retrieve_fee'][0]
        collaborator_data[device]['send_fee'] = collaborator_data[device]['send_fee'][0]

    manager_data['send_aggregated_weights_fee'] = [manager_data['send_aggregated_weights_fee'][0],manager_data['send_aggregated_weights_fee'][1]]

    collaborator_df = pd.DataFrame.from_dict(collaborator_data, orient='index').T
    manager_df = pd.DataFrame.from_dict(manager_data, orient='index').T
    name_file = collaborator_file.split('_')
    # Print DataFrames
    print(f"\t\tFees for {name_file[-3]} devices and {name_file[-2]} out of battery:\n")
    print("Collaborators:\n",collaborator_df)
    print()
    print("Manager:\n",manager_df)
    print("\n" + "=" * 40 + "\n")


		Fees for 3 devices and [] out of battery:

Collaborators:
                       A       B       C
retrieve_fee      28941   31532   34123
send_fee         155983  128574  133061
model_start_fee  150325  125507  134469

Manager:
   deploy_mocks_fee open_blockchain_fee add_collaborator_fee  \
0          2151147               28934               165954   

  send_aggregated_weights_fee send_model_fee change_state_fee  
0              [74405, 49805]         192543           357957  


		Fees for 4 devices and [] out of battery:

Collaborators:
                       A       B       C       D
retrieve_fee      28941   31532   34123   36714
send_fee         155983  128574  131165  135652
model_start_fee  150325  125507  130689  139651

Manager:
   deploy_mocks_fee open_blockchain_fee add_collaborator_fee  \
0          2151147               28934               216260   

  send_aggregated_weights_fee send_model_fee change_state_fee  
0              [83795, 59195]         192543           4